# Перекрёстная валидация

В этой тетради мы изучим метод кросс-валидации для оценки производительности 
моделей машинного обучения и понять его преимущества по сравнению  
простым разделением данных на обучающий и тестовый наборы.

***

## Знакомство с кросс-валидацией

Рассмотрим, что из себя вообще представляет кросс-валидация в общих чертах, а затем рассмотрим конкретные интерфейсы, поставляемые фреймворком Scikit-Learn.

Перекрестная валидация - это статистический метод, используемый для оценки и сравнения моделей машинного обучения путем разделения доступных данных на два сегмента: один для обучения модели, а другой для проверки модели. Цель перекрестной проверки - оценить, насколько хорошо модель обобщается на новых данных, и выявить потенциальные проблемы, такие как переобучение.

Можно выделить два типа перекрестной проверки:
* Исчерпывающая перекрестная проверка: Этот метод изучает и тестирует все возможные способы разделения исходной выборки на обучающий и проверочный набор.
* K-кратная перекрестная проверка: При k-кратной перекрестной проверке исходная выборка случайным образом разбивается на $k$ подвыборок одинакового размера. Из $k$ подвыборок одна подвыборка сохраняется в качестве проверочных данных для тестирования модели, а остальные $k - 1$ подвыборок используются в качестве обучающих данных. Затем процесс перекрестной проверки повторяется $k$ раз, причем каждая из $k$ подвыборок используется ровно один раз в качестве данных для проверки.

Таким образом, перекрестная проверка является ценным методом оценки моделей машинного обучения, гарантирующим, что модель хорошо обобщает, а не просто запоминает обучающие данные. Это надежный способ оценить производительность модели на данных, которые модель до этого не видела, и широко используется в машинном обучении.

Перейдём теперь к конкретным методам, которые мы можем применить к нашим данным.

### K-Fold

K-Fold (`KFold`) делит все выборки на $k$ групп выборок, называемые складками (если $k = n$, это эквивалентно стратегии исключения одной выборки), одинакового размера (если это возможно). Функция прогнозирования изучается с $k - 1$ помощью складок, а оставшаяся часть используется для тестирования.

### Stratified K-Fold

Стратифицированный K-Fold - это вариация `KFold`, которая возвращает стратифицированные складки: каждый набор содержит примерно такой же процент образцов каждого целевого класса, как и полный набор.

### Stratified Shuffle Split

Этот метод суть вариация [`ShuffleSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit), которая возвращает стратифицированные разбиения, т.е. создает разбиения, сохраняя тот же процент для каждого целевого класса, что и в полном наборе.

Подробнее об этих и других методах кросс-валидации можно почитать [здесь](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-and-model-selection).

Приступим теперь к практическим примерам.

***

## Практические примеры

Посмотрим сейчас, как можно применять вышеописанные методы.

Однако сначала, как обычно, выполним импорты.

In [1]:
from typing import Optional

from numpy import ndarray
from pandas import read_csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    KFold,
    StratifiedKFold,
    StratifiedShuffleSplit,
    train_test_split,
)

А так же выполним загрузку данных. В этот раз воспользуемся набором данных ирисов Фишера.

In [2]:
iris = read_csv("../datasets/iris.csv")

iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


Отделим независимые пременные от целевой переменной.

In [3]:
x_iris, y_iris = iris.iloc[:, 1:-1], iris.iloc[:, -1]

### K-Fold

Первым делом рассмотрим процесс работы `KFold`. Предварительно инициализируем модель Случайного леса.

In [4]:
def rate_accuracy(
    x: ndarray,
    y: ndarray,
    title: str,
    *,
    validator,
    shuffle: Optional[bool] = None,
    n_splits: int,
) -> None:
    random_forest = RandomForestClassifier()
    validator_initialized = validator(n_splits=n_splits)
    if shuffle:
        validator_initialized = validator(n_splits=n_splits, shuffle=True, random_state=52)
    kfold_score = cross_val_score(
        random_forest, x, y,
        cv=validator_initialized,
        scoring="accuracy",
    )

    print("%s: %0.2f (+/- %0.2f)" % (title, kfold_score.mean(), kfold_score.std() * 2))


rate_accuracy(x_iris, y_iris, "K-Fold accuracy", validator=KFold, shuffle=True, n_splits=5)

K-Fold accuracy: 0.96 (+/- 0.07)


Посмотрим сейчас, как на точность повлияет количество разбиений.

In [5]:
rate_accuracy(
    x_iris, y_iris, "K-Fold accuracy with 15 splits",
    shuffle=True, validator=KFold, n_splits=15,
)
rate_accuracy(
    x_iris, y_iris, "K-Fold accuracy with 10 splits",
    shuffle=True, validator=KFold, n_splits=10,
)
rate_accuracy(
    x_iris, y_iris, "K-Fold accuracy with 3 splits",
    shuffle=True, validator=KFold, n_splits=3,
)

K-Fold accuracy with 15 splits: 0.95 (+/- 0.12)
K-Fold accuracy with 10 splits: 0.95 (+/- 0.09)
K-Fold accuracy with 3 splits: 0.94 (+/- 0.07)


Как видим, точность выше, когда разбиений больше, однако прямо пропорционально количеству разбиений растёт и отклонение. 

### Stratified K-Fold

Приступим к `StratifiedKFold` и выясним, сильно ли нам поможет стратификация.

In [6]:
rate_accuracy(
    x_iris, y_iris, "Stratified K-Fold accuracy with 15 splits",
    shuffle=True, validator=StratifiedKFold, n_splits=15,
)
rate_accuracy(
    x_iris, y_iris, "Stratified K-Fold accuracy with 10 splits",
    shuffle=True, validator=StratifiedKFold, n_splits=10,
)
rate_accuracy(
    x_iris, y_iris, "Stratified K-Fold accuracy with 3 splits",
    shuffle=True, validator=StratifiedKFold, n_splits=3,
)

Stratified K-Fold accuracy with 15 splits: 0.95 (+/- 0.14)
Stratified K-Fold accuracy with 10 splits: 0.96 (+/- 0.14)
Stratified K-Fold accuracy with 3 splits: 0.94 (+/- 0.07)


В общем, точность не изменилась, но отклонение стало меньше при пятнадцати разбиениях.

### Stratified Shuffle Split

И, наконец, проанализируем результаты, которые нам выдаст `StratifiedShuffleSplit`.

In [7]:
rate_accuracy(
    x_iris, y_iris, "Stratified Shuffle Split accuracy with 15 splits",
    validator=StratifiedShuffleSplit, n_splits=15,
)
rate_accuracy(
    x_iris, y_iris, "Stratified Shuffle Split accuracy with 10 splits",
    validator=StratifiedShuffleSplit, n_splits=10,
)
rate_accuracy(
    x_iris, y_iris, "Stratified Shuffle Split accuracy with 3 splits",
    validator=StratifiedShuffleSplit, n_splits=3,
)

Stratified Shuffle Split accuracy with 15 splits: 0.94 (+/- 0.11)
Stratified Shuffle Split accuracy with 10 splits: 0.95 (+/- 0.12)
Stratified Shuffle Split accuracy with 3 splits: 1.00 (+/- 0.00)


Интересно. В этот раз отклонение не так велико при большем количестве разбиений, однако становится намного сильнее при меньшем их количестве по сравнению с другмими методами.

***

## Оптимизация моделей

Теперь, когда мы уже знакомы с некоторыми методами кросс-валидации, попробуем оптимизировать гиперпараметры модели Случайного леса, которой мы пользовались выше. Для этого нам пригодится класс [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV). Однако сперва нужно разбить данные на тренировочную и обучающую выборки. Масштабирование делать необязательно, так как деревья решений склонны показывать лучший результат на сырых данных.

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x_iris, y_iris, test_size=0.3, random_state=52)
# Defining the parameters variations
parameters_grid = {
    "n_estimators": (10, 100, 1000),
    "criterion": ("gini", "entropy", "log_loss"),
    "max_depth": (10, 100, None),
    "min_samples_split": (2, 4, 6),
    "min_samples_leaf": (1, 2, 3),
    "min_weight_fraction_leaf": (0.1, 0.01, 0.0),
}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid=parameters_grid)

# Fitting the optimizer with the training data
grid_search.fit(x_train, y_train)
# Obtaining the best parameters for the model
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 100,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10}

В результате мы получили параметры для модели, которые гарантируют следующую точность.

In [9]:
grid_search.score(x_test, y_test)

0.9111111111111111

Очень даже неплохо.

Подведём итоги.

***

# Заключение

В этой тетради мы научились пользоваться алгоритмами перекрёстной провекрки. Выполнив код, демонстрирующий работу различных алгоритмов перекрёстной проверки в «игрушеченой» среде, мы увидели значимость их влада в точность моделей даже на небольших наборах данных.

Стоит ещё раз отметить, что перекрестная проверка является ценным методом оценки моделей машинного обучения, гарантирующим, что модель хорошо обобщает, а не просто запоминает обучающие данные. Это надежный способ оценить производительность модели на данных, которые модель до этого не видела, и широко используется в машинном обучении.